In [28]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader

device = T.device("cuda:0" if T.cuda.is_available() else "cpu")
ACTIONS= ["IDLE", "L1", "L2", "L3"]
GB2MB=1000

In [29]:

class DuelingDeepQNetwork(nn.Module):
    def __init__(self, alpha, state_dim, action_dim, fc1_dim, fc2_dim):
        super(DuelingDeepQNetwork, self).__init__()

        self.fc1 = nn.Linear(state_dim, fc1_dim)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.V = nn.Linear(fc2_dim, 1)
        self.A = nn.Linear(fc2_dim, action_dim)

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.to(device)

    def forward(self, state):
        x = T.relu(self.fc1(state))
        x = T.relu(self.fc2(x))

        V = self.V(x)
        A = self.A(x)
        Q = V + A - T.mean(A, dim=-1, keepdim=True)

        return Q

    def save(self, file_path):
        T.save(self, file_path)

cache_agent = DuelingDeepQNetwork(alpha=1e-4,
                              state_dim=15,
                              action_dim=len(ACTIONS),
                              fc1_dim=64,
                              fc2_dim=32)


In [30]:


class CacheAgentDataset(Dataset):
    def __init__(self):
        self.data = []
        self.labels = []
        self.generate()


    def gen(self,func, num=1000):
        for _ in range(num):
            obs, label = func()
            self.data.append(obs)
            self.labels.append(label)

    def generate(self):
        self.gen(self.cold_start)

    def cold_start(self):
        obs = {
            "es_load":random.uniform(0, 0.1),
            "free_storage_size_ratio_L1":random.uniform(0, 0.1),
            "free_storage_size_ratio_L2":random.uniform(0.9, 1),
            "free_storage_size_ratio_L3":random.uniform(0.9, 1),
            "can_L1_fit":1,
            "can_L2_fit":1,
            "can_L3_fit":1,
            "servie_size":random.uniform(5, 1000)/GB2MB,
            "estimated_fetch_time":random.uniform(1, 60),
            "is_popular":0,
            "charm":round(abs(np.random.standard_normal(1)[0]), 2),
            "service_request_frequency":random.randint(0, 3),
            "nearby_servers_count":random.uniform(0, 1),
            "cached_in_nearby_servers":0,
            "es_request_frequency":random.uniform(0, 1)
        }
        return obs, "L1"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]



In [31]:
import pandas as pd

class SimulatedDataset(Dataset):
    def __init__(self):
        self.data = []
        self.labels = []
        self.load_data()
    
    def load_data(self):
        filepath = "../../cache_df.csv"
        df = pd.read_csv(filepath)
        for i in range(len(df)):
            obs = df.iloc[i, 0:-1].to_dict()
            label = df.iloc[i, -1]
            self.data.append(obs)
            self.labels.append(label)
        print(self.data[0])
        print(self.labels[0])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        obs = list(self.data[index].values())
        label = self.labels[index]
        label_index = ACTIONS.index(label)
        one_hot = np.zeros(len(ACTIONS))
        one_hot[label_index] = 1
        return T.tensor(obs).float(), T.tensor(one_hot).float()

def train(network, epochs):
    loss_func = nn.CrossEntropyLoss()
    dataset = SimulatedDataset()
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    optimizer = network.optimizer
    for i in range(epochs):
        for data, labels in dataloader:
            optimizer.zero_grad()
            output = network(data)
            loss = loss_func(output, labels)
            loss.backward()
            optimizer.step()
            print(loss)

if __name__ == "__main__":
    train(cache_agent, 10)
    cache_agent.save("cache_agent.pth")

{'es_load': 0.0758620689655172, 'free_storage_size_ratio_L1': 1.0, 'free_storage_size_ratio_L2': 1.0, 'free_storage_size_ratio_L3': 1.0, 'can_L1_fit': True, 'can_L2_fit': True, 'can_L3_fit': True, 'service_size': 8.029, 'estimated_fetch_time': 0.1128421757506457, 'is_popular': False, 'charm': 2.2, 'service_request_frequency': 0.0, 'nearby_servers_count': 3, 'cached_in_nearby_servers': False, 'es_request_frequency': 0.001}
L1
tensor(34.0881, grad_fn=<DivBackward1>)
tensor(20.2368, grad_fn=<DivBackward1>)
tensor(26.0727, grad_fn=<DivBackward1>)
tensor(30.8356, grad_fn=<DivBackward1>)
tensor(19.8314, grad_fn=<DivBackward1>)
tensor(30.5379, grad_fn=<DivBackward1>)
tensor(22.9332, grad_fn=<DivBackward1>)
tensor(32.5653, grad_fn=<DivBackward1>)
tensor(34.4222, grad_fn=<DivBackward1>)
tensor(24.9237, grad_fn=<DivBackward1>)
tensor(35.4099, grad_fn=<DivBackward1>)
tensor(24.7692, grad_fn=<DivBackward1>)
tensor(27.3522, grad_fn=<DivBackward1>)
tensor(20.3927, grad_fn=<DivBackward1>)
tensor(20.5